In [9]:
import sqlite3
import scipy.stats as stats
import pandas as pd
import numpy as np

Câu 1: Viết câu lệnh SQL tính tương quan giữa A và B

In [4]:

# Tạo dataframe mẫu
data = {
    'A': [8, 7.5, 6, 7],
    'B': [9, 8.5, 7, 6]
}
df = pd.DataFrame(data)

# Lưu vào SQLite
conn = sqlite3.connect(':memory:')
df.to_sql('Scores', conn, index=False, if_exists='replace')

# Viết truy vấn SQL để lấy các phần cần thiết
query = """
SELECT 
    COUNT(*) as n,
    SUM(A*B) as sum_ab,
    SUM(A) as sum_a,
    SUM(B) as sum_b,
    SUM(A*A) as sum_aa,
    SUM(B*B) as sum_bb
FROM Scores
"""
result = pd.read_sql(query, conn)

# Tính Pearson theo công thức thủ công
n = result['n'][0]
sum_ab = result['sum_ab'][0]
sum_a = result['sum_a'][0]
sum_b = result['sum_b'][0]
sum_aa = result['sum_aa'][0]
sum_bb = result['sum_bb'][0]

numerator = (n * sum_ab) - (sum_a * sum_b)
denominator = ((n * sum_aa - sum_a**2) * (n * sum_bb - sum_b**2))**0.5
r = numerator / denominator
print(f"Hệ số tương quan r = {r:.4f}")


Hệ số tương quan r = 0.6911


Câu 2:

In [5]:
df_long = pd.DataFrame({
    'Day': ['Day 1']*3 + ['Day 2']*3 + ['Day 3']*3 + ['Day 4']*3,
    'Model': ['A', 'B', 'C']*4,
    'Score': [8, 9, 7, 7.5, 8.5, 7, 6, 7, 8, 7, 6, 5]
})


In [6]:


# Pivot lại để đưa về dạng bảng đếm
contingency = pd.pivot_table(df_long, values='Score',
                              index='Day', columns='Model',
                              aggfunc='mean')

# Vì là kiểm định độ phân phối điểm số → ANOVA hoặc Kruskal-Wallis phù hợp hơn:
grouped = df_long.groupby('Model')['Score'].apply(list)
stat, p = stats.f_oneway(*grouped)

print(f"ANOVA p-value: {p}")


ANOVA p-value: 0.5954382115847785


Câu 3:

In [7]:

# Ví dụ dữ liệu
df = pd.DataFrame({'departure_time': [830, 1445]})

# Chuyển đổi về dạng datetime
df['formatted_time'] = pd.to_datetime(df['departure_time'].astype(str).str.zfill(4), format='%H%M').dt.time
print(df)


   departure_time formatted_time
0             830       08:30:00
1            1445       14:45:00


 Câu 4:

In [10]:


# Giả sử bảng "X" có cột "value"
df = pd.DataFrame({'value': [10, 11, 9, 50, 10, 11, 12]})
median = df['value'].median()
mad = np.median(np.abs(df['value'] - median))

# Giá trị ngoại lệ nếu lệch > 1.5 * MAD
threshold = 1.5 * mad
df['is_outlier'] = np.abs(df['value'] - median) > threshold
print(df)


   value  is_outlier
0     10       False
1     11       False
2      9        True
3     50        True
4     10       False
5     11       False
6     12       False


Câu 5

In [11]:
df = pd.DataFrame({
    'last_name': ['Nguyen', 'Nguyen', 'Tran'],
    'weight': [60, 61, 55],
    'height': [170, 170, 165]
})

# Kiểm tra từng cặp có trùng tên và chênh lệch cân <= 2kg
from itertools import combinations

def is_same_person(row1, row2):
    return (row1['last_name'] == row2['last_name']) and (abs(row1['weight'] - row2['weight']) <= 2)

for i, j in combinations(df.index, 2):
    if is_same_person(df.loc[i], df.loc[j]):
        print(f"{i} và {j} có thể là cùng một người.")

0 và 1 có thể là cùng một người.
